In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

2024-07-05 14:20:16.414131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 14:20:16.414293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 14:20:16.584503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir = '/kaggle/input/cat-and-dog/training_set/training_set'
test_dir = '/kaggle/input/cat-and-dog/test_set/test_set'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Adding validation split
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
batch_size = 32
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 6404 images belonging to 2 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 1601 images belonging to 2 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 2023 images belonging to 2 classes.


In [8]:
svm_classifier = SVC(kernel='linear')

In [9]:
train_images = []
train_labels = []
num_train_batches = len(train_generator)
for _ in range(num_train_batches):
    batch_images, batch_labels = next(train_generator)
    train_images.append(batch_images)
    train_labels.append(batch_labels)
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

In [10]:
validation_images = []
validation_labels = []
num_validation_batches = len(validation_generator)
for _ in range(num_validation_batches):
    batch_images, batch_labels = next(validation_generator)
    validation_images.append(batch_images)
    validation_labels.append(batch_labels)
validation_images = np.concatenate(validation_images)
validation_labels = np.concatenate(validation_labels)

In [11]:
train_images = train_images.reshape(train_images.shape[0], -1)
validation_images = validation_images.reshape(validation_images.shape[0], -1)

In [12]:
svm_classifier.fit(train_images, train_labels)

SVC(kernel='linear')

In [13]:
validation_preds = svm_classifier.predict(validation_images)
print("Validation Accuracy:", np.mean(validation_preds == validation_labels))
print("\nValidation Classification Report:")
print(classification_report(validation_labels, validation_preds))
print("\nValidation Confusion Matrix:")
print(confusion_matrix(validation_labels, validation_preds))

Validation Accuracy: 0.5421611492816989

Validation Classification Report:
              precision    recall  f1-score   support

         0.0       0.54      0.59      0.56       800
         1.0       0.55      0.50      0.52       801

    accuracy                           0.54      1601
   macro avg       0.54      0.54      0.54      1601
weighted avg       0.54      0.54      0.54      1601


Validation Confusion Matrix:
[[471 329]
 [404 397]]


In [14]:
test_images = []
test_labels = []
num_test_batches = len(test_generator)
for _ in range(num_test_batches):
    batch_images, batch_labels = next(test_generator)
    test_images.append(batch_images)
    test_labels.append(batch_labels)
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

In [15]:
test_images = test_images.reshape(test_images.shape[0], -1)

In [16]:
test_preds = svm_classifier.predict(test_images)
print("Test Accuracy:", np.mean(test_preds == test_labels))
print("\nClassification Report on Test Set:")
print(classification_report(test_labels, test_preds))

Test Accuracy: 0.5462184873949579

Classification Report on Test Set:
              precision    recall  f1-score   support

         0.0       0.54      0.59      0.57      1011
         1.0       0.55      0.50      0.52      1012

    accuracy                           0.55      2023
   macro avg       0.55      0.55      0.55      2023
weighted avg       0.55      0.55      0.55      2023



In [17]:
print("\nConfusion Matrix on Test Set:")
print(confusion_matrix(test_labels, test_preds))


Confusion Matrix on Test Set:
[[599 412]
 [506 506]]
